# Pseudocode

[[Previously](./01-buggy-sieve.ipynb) I wrote a buggy implementation of the Sieve of Eratosthenes. You may want to read that first.]

It can be helpful to represent algorithms in a notation that is more precise than the written word, 
and not specific to a particular programming language.

It turns out that the [Wikipedia page](https://en.wikipedia.org/wiki/Sieve_of_Eratosthenes) for the _Sieve of Eratosthenes_ includes a pseudocode representation.

```
algorithm Sieve of Eratosthenes is
    input: an integer n > 1.
    output: all prime numbers from 2 through n.

    let A be an array of Boolean values, indexed by integers 2 to n,
    initially all set to true.
    
    for i = 2, 3, 4, ..., not exceeding √n do
        if A[i] is true
            for j = i^2, i^2+i, i^2+2i, i^2+3i, ..., not exceeding n do
                A[j] := false

    return all i such that A[i] is true.
```

The _circumflex_ '^' is a way of saying _to the power of_.

I knew in my head the principle of how this works -- create an array and mark successive factors as non-prime -- and 
in my enthusiasm to code it I overlooked the helpful detail here. There's a lesson there ... do some due diligence first before launching in. Someone else may have done some work that makes your job easier.

So let's revisit our example.

I'm going to take a slightly different approach and start with the assertion of what our expected behaviour is, as a function.

In [ ]:
function primesUpTo(limit) {
    return [];
}

In [ ]:
primesUpTo(20) === [2,3,5,7,11,17,19]

I've defined a function that takes the (integer) limit as a parameter and returns an array. 
Let's start with a failing test. And implement the algorithm.

In [1]:
function primesUpTo(limit) {
    const maxPossibleFactor = Math.floor(Math.sqrt(limit));
    console.log(`maxPossibleFactor ${maxPossibleFactor}`);
    let possiblePrimes = new Array(limit-1); // Represents numbers 2 to n
    possiblePrimes.fill(true);
    for(let i=2; i <= maxPossibleFactor; i++) {
        console.log(`i ${i} possiblePrime ${possiblePrimes[i-2]}`);
        if (possiblePrimes[i-2]) {
            for(let k=0; i*(i+k) <= limit; k++) {
                let j = i*(i+k);
                possiblePrimes[j-2] = false; 
            }
        }
    }
    console.log(`possiblePrimes ${possiblePrimes}`);
    const actualPrimes = [];
    for(let i=0; i < possiblePrimes.length; i++) {
        if (possiblePrimes[i]) {
            actualPrimes.push(i+2);
        }
    }
    return actualPrimes;
}

The first version I wrote didn't work:

```
function primesUpTo(limit) {
    const maxPossibleFactor = Math.floor(Math.sqrt(limit));
    let possiblePrimes = new Array(limit-1); // Represents numbers 2 to n
    possiblePrimes.fill(true);
    for(let i=2; i <= maxPossibleFactor; i++) {
        if (possiblePrimes[i-2]) {
            for(let k=0, j=i*(i+k); j <= limit; k++) {
                possiblePrimes[j-2] = false; 
            }
        }
    }
    const actualPrimes = [];
    for(let i=0; i < possiblePrimes.length; i++) {
        if (possiblePrimes[i]) {
            actualPrimes.push(i+2);
        }
    }
    return actualPrimes;
}
```

So I added the console.log statements and noticed that the second one wasn't being reached. 
That suggest a problem with my loop conditions.

Adding a print statement just inside the outer loop (which sets _i) I see this:
```
maxPossibleFactor 4
i 2 possiblePrime true
```

and I can't run any more code which suggests the code is not returning. (I had to use the _Kernel_ menu to restart it.) There's a problem with that inner _for_ loop.

[Eureka](https://en.wikipedia.org/wiki/Eureka_(word)) (as Eratosthenes may have said once or twice)! I'm only initialising the j once at the start of the loop. It's the middle condition that is evaluated each time around the loop so I need to recalculate j there. 

In [2]:
primesUpTo(20)

maxPossibleFactor 4
i 2 possiblePrime true
i 3 possiblePrime true
i 4 possiblePrime false
possiblePrimes true,true,false,true,false,true,false,false,false,true,false,true,false,false,false,true,false,true,false


[
   2,  3,  5,  7,
  11, 13, 17, 19
]

That looks sensible. Let's try our test again.

In [3]:
primesUpTo(20) === [2,3,5,7,11,17,19]

maxPossibleFactor 4
i 2 possiblePrime true
i 3 possiblePrime true
i 4 possiblePrime false
possiblePrimes true,true,false,true,false,true,false,false,false,true,false,true,false,false,false,true,false,true,false


false

Hmm. Still false. Oh wait. My test data was wrong. Tests are useful, but they need to test for the right thing! 
That's why we review each other's code in teams to spot things that we may have missed.

In [5]:
primesUpTo(20) == [2,3,5,7,11,13,17,19]

maxPossibleFactor 4
i 2 possiblePrime true
i 3 possiblePrime true
i 4 possiblePrime false
possiblePrimes true,true,false,true,false,true,false,false,false,true,false,true,false,false,false,true,false,true,false


false

Still false. :( It turns out that JavaScript doesn't support comparing arrays using the double equals notation. 
(Single equals is used for variable assignment.) So let's write a helper function to compare arrays.

In [8]:
function arrayEquals(a, b) {
    if (a.length != b.length) {
        return false;
    }
    for(let i=0; i < a.length; i++) {
        if (a[i] != b[i]) {
            return false;
        }
    }
    return true;
}

In [9]:
arrayEquals(primesUpTo(20), [2,3,5,7,11,13,17,19])

maxPossibleFactor 4
i 2 possiblePrime true
i 3 possiblePrime true
i 4 possiblePrime false
possiblePrimes true,true,false,true,false,true,false,false,false,true,false,true,false,false,false,true,false,true,false


true

Now we're in business.

Let's try something more challenging. (I'm going to comment out print statements first.)

In [11]:
function primesUpTo(limit) {
    const maxPossibleFactor = Math.floor(Math.sqrt(limit));
    let possiblePrimes = new Array(limit-1); // Represents numbers 2 to n
    possiblePrimes.fill(true);
    for(let i=2; i <= maxPossibleFactor; i++) {
        if (possiblePrimes[i-2]) {
            for(let k=0; i*(i+k) <= limit; k++) {
                let j = i*(i+k);
                possiblePrimes[j-2] = false; 
            }
        }
    }
    const actualPrimes = [];
    for(let i=0; i < possiblePrimes.length; i++) {
        if (possiblePrimes[i]) {
            actualPrimes.push(i+2);
        }
    }
    return actualPrimes;
}

In [13]:
const primesTo2000000 = primesUpTo(2000000)

In [14]:
primesTo2000000.length

148933

It [turns out](https://primes.utm.edu/howmany.html) you can approximate the number of primes below a particular number. But that doesn't tell us if our answer is right.

The last number is indeed the largest prime below 2000000.

In [19]:
primesTo2000000[primesTo2000000.length-1]

1999993

In [22]:
// This is a comment. Let's do some limit checking ...
primesUpTo(0)

RangeError: Invalid array length

If this code were to become part of a long-lived project that other people might unknowlingly call in unintended ways, 
we should tighten up the rules on what is and isn't allowed. For example, what would primesUpTo(2.3) return? Or negative numbers. 

And what about large numbers? This function creates an array that is close enough to the same size as the limit, and that is stored in memory. If we make that number big enough it is going to consume a large amount of memory which can either degrade the behaviour of the programme or make it fail outright. The Wikipedia page suggests some alternative algorithms that mitigate this issue somewhat.

So some sensible checks might include a minimum and maximum value for the limit, and checking that the limit is a whole number. 

What would happen if we called primesUpTo with a string argument like this ... ?

In [25]:
primesUpTo("20")

[
   2,  3,  5,  7,
  11, 13, 17, 19
]

Miraculously it worked because JavaScript tried to make sense of what we were doing, but I wouldn't have expected that.

When designing algorithms we are trading off storage and time requirements. The [time complexity](https://www.geeksforgeeks.org/how-is-the-time-complexity-of-sieve-of-eratosthenes-is-nloglogn/) of the basic algorithm is the order of n*log(log(n)) and the space complexity is order of n, or O(n).

On a final point, it's worth noticing the optimisation in the code which checks `if (possiblePrimes[i-2]) {` and avoids unnecessary work by ignoring multiples that we've already flagged. For example, all multiples of 2 are non-prime, which means that all multiples of 4 are non-prime. And so on. So if we find a number that has already been marked as non-prime we don't need to do any further processing for that number.

**Postscript** I'm aware that some people may look at this code and say _really, you made a lot of rookie errors in this example, you must be a pretty bad coder_. To which I would reply:

* I don't think it's very helpful to present the finished article as though you arrived at that point the first time. This suggests that most people get from start to finish on a linear path with minimal errors, and I don't believe that's the case. It may be for some people, but not most.

* It's OK not to be right first time; figuring out where you're wrong by breaking the problem down into smaller parts is a helpful way to find where you may be making an invalid assumption, or were distracted by the sound of the washing machine.

* This tendency in our industry to present our finished work and not revealing our mis-steps can lead to feelings of inadequacy in those who are learning, and I think we can do better than that.

I hope this notebook has highlighted a few points, namely:

* doing some up-front due diligence with reliable sources can save you time

* writing tests is useful; writing correct tests is even better

* it's a good idea to understand the [complexity](https://www.freecodecamp.org/news/the-complexity-of-simple-algorithms-and-data-structures-in-javascript-11e25b29de1e/) of updating and reading data structures in your particular language 